<a href="https://colab.research.google.com/github/shima2920/Kaggle/blob/main/Recruit_Restaurant_Visitor_Forecasting_20210330.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#https://www.kaggle.com/general/74235

! pip install -q kaggle

from google.colab import files
files.upload()
#Kaggle account で発行した API token key ファイルを Upload する。

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

! kaggle competitions download -c recruit-restaurant-visitor-forecasting

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              221KB  2021-03-30 07:16:41           1285  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-02-17 16:10:44            938  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27            774  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:1

# This is Kaggle Competition "Recruit Restaurant Visitor Forecasting"



https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting

ref code :
https://www.kaggle.com/meli19/py-single-light-gbm-lb-0-521

# Summary



*   LightGBM with optuna

        Accuracy = 0.5248954241110735 (2021/03/30)
        Private Score : 0.56076



*   Chenge the method of removing the outlier

        Accuracy = 0.5270059188240132 (2021/03/30)
        Private Score : 0.55330





EDA

Improve preprocessing

Try another model


In [108]:
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder#Label Encoding
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb #LightLGBのハイパーパラメータチューニングモジュールであるOptunaをimport
import lightgbm as lgb_original
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import xgboost as XGBClassifier
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Dropout #Dense : ニューラルネットの層を定義？
from keras.models import Sequential# 複雑なニューラルネットワークモデルを簡略化するためのモデルの一つであるSequentialモデル
from sklearn.preprocessing import StandardScaler #標準化



#Validation
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold


#イテレータ, ループ処理を実行してくれる
import itertools

pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_columns', 1000)
%matplotlib inline

In [109]:
print("Loading Data ...")
# air_reserve
df_ar = pd.read_csv('air_reserve.csv.zip')
# air_store_info
df_as = pd.read_csv('air_store_info.csv.zip')
# air_visit_data
df_av = pd.read_csv('air_visit_data.csv.zip')
# hpg_reserve
df_hr = pd.read_csv('hpg_reserve.csv.zip')
# hpg_store_info
df_hs = pd.read_csv('hpg_store_info.csv.zip')
# date_info
df_di = pd.read_csv('date_info.csv.zip')
# sample_submission
df_ss = pd.read_csv('sample_submission.csv.zip')
# store_id_relation
df_si = pd.read_csv('store_id_relation.csv.zip')

Loading Data ...


In [110]:
# df_test
df_test = pd.read_csv('sample_submission.csv.zip')
df_test['air_store_id'] = df_test['id'].apply(lambda x: '_'.join(x.split('_')[:2]))
df_test['visit_date'] = df_test['id'].apply(lambda x: x.split('_')[-1])
index_test = df_test['id']
del df_test['id'], df_test['visitors']

In [111]:
import gc
gc.collect()
print("Loading Data Compelete.")


Loading Data Compelete.


In [112]:
print("=========================================================================================")
print("Data Exploring ...")
print("=========================================================================================")
print("Unique store id in different dataset :")
print("-----------------------------------------------------------------------------------------")
num_store_ar = np.unique(df_ar['air_store_id'])
print("Number of unique stores in 'df_ar' is:" + str(len(num_store_ar)))

num_store_as = np.unique(df_as['air_store_id'])
print("Number of unique stores in 'df_as' is:" + str(len(num_store_as)))

num_store_av = np.unique(df_av['air_store_id'])
print("Number of unique stores in 'df_av' is:" + str(len(num_store_av)))

print("-----------------------------------------------------------------------------------------")
num_store_in_hr = np.unique(df_hr['hpg_store_id'])
print("Number of unique stores in 'df_hr' is:" + str(len(num_store_in_hr)))

num_store_in_hs = np.unique(df_hs['hpg_store_id'])
print("Number of unique stores in 'df_hs' is:" + str(len(num_store_in_hs)))

print("-----------------------------------------------------------------------------------------")
num_store_in_test = np.unique(df_test['air_store_id'])
print("Number of unique stores in 'df_test' is:" + str(len(num_store_in_test)))

print("-----------------------------------------------------------------------------------------")
num_store_in_si = np.unique(df_si['air_store_id'])
print("Number of unique stores in 'df_test' is:" + str(len(num_store_in_si)))
print("=========================================================================================")


Data Exploring ...
Unique store id in different dataset :
-----------------------------------------------------------------------------------------
Number of unique stores in 'df_ar' is:314
Number of unique stores in 'df_as' is:829
Number of unique stores in 'df_av' is:829
-----------------------------------------------------------------------------------------
Number of unique stores in 'df_hr' is:13325
Number of unique stores in 'df_hs' is:4690
-----------------------------------------------------------------------------------------
Number of unique stores in 'df_test' is:821
-----------------------------------------------------------------------------------------
Number of unique stores in 'df_test' is:150


In [113]:
'''
# =============================================================================
# remove outliers
# =============================================================================
def remove_outliers(data):
    df_0 = data.loc[data.visitors == 0]   
    q1 = np.percentile(data.visitors, 25, axis=0)
    q3 = np.percentile(data.visitors, 75, axis=0)
#    k = 5
#    k = 2.5
    k = 2.8
#    k = 2
#    k = 1.5
    iqr = q3 - q1
    df_temp = data.loc[data.visitors > q1 - k*iqr]
    df_temp = data.loc[data.visitors < q3 + k*iqr]
    frames = [df_0, df_temp]
    result = pd.concat(frames)
    return result

df_av = remove_outliers(df_av)
'''

'\n# =============================================================================\n# remove outliers\n# =============================================================================\ndef remove_outliers(data):\n    df_0 = data.loc[data.visitors == 0]   \n    q1 = np.percentile(data.visitors, 25, axis=0)\n    q3 = np.percentile(data.visitors, 75, axis=0)\n#    k = 5\n#    k = 2.5\n    k = 2.8\n#    k = 2\n#    k = 1.5\n    iqr = q3 - q1\n    df_temp = data.loc[data.visitors > q1 - k*iqr]\n    df_temp = data.loc[data.visitors < q3 + k*iqr]\n    frames = [df_0, df_temp]\n    result = pd.concat(frames)\n    return result\n\ndf_av = remove_outliers(df_av)\n'

In [114]:
###上位〇〇%, 下位△△%をulimit/llimitの値に置き換える(外れ値の除去)####################################
for col in ["visitors"]:
    ulimit = np.percentile(df_av[col].values, 99.5)
    llimit = np.percentile(df_av[col].values, 0.5)
    df_av[col].iloc[df_av[col]>ulimit] = ulimit
    df_av[col].iloc[df_av[col]<llimit] = llimit

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [115]:
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [116]:
# =============================================================================
# df to dict for mapping and dropping
# =============================================================================
print('mapping and dropping useless information in df_hr ...')
s_1 = df_si['air_store_id']
s_2 = df_si['hpg_store_id']
a_h_map = dict(zip(s_2.values, s_1.values))
del s_1, s_2

df_hr['air_store_id'] = df_hr['hpg_store_id'].map(a_h_map)
df_hr = df_hr.drop('hpg_store_id', axis=1).dropna()


print('mapping and dropping useless information in df_hr Done!')
print("-----------------------------------------------------------------------------------------")

print('mapping and dropping useless information in df_hr ...')

df_hs['air_store_id'] = df_hs['hpg_store_id'].map(a_h_map)
df_hs = df_hs.drop('hpg_store_id', axis=1).dropna()
print('mapping and dropping useless information in df_hs Done!')
gc.collect()
print("=========================================================================================")


mapping and dropping useless information in df_hr ...
mapping and dropping useless information in df_hr Done!
-----------------------------------------------------------------------------------------
mapping and dropping useless information in df_hr ...
mapping and dropping useless information in df_hs Done!


In [117]:
df_hr.head()

,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
103,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
121,2016-01-01 19:00:00,2016-01-01 16:00:00,2,air_08cb3c4ee6cd6a22
272,2016-01-02 17:00:00,2016-01-01 22:00:00,3,air_6b15edd1b4fbb96a
348,2016-01-02 18:00:00,2016-01-02 17:00:00,2,air_37189c92b6c761ec
349,2016-01-02 18:00:00,2016-01-01 20:00:00,2,air_37189c92b6c761ec


In [118]:
df_av.head(5)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25.0
1,air_ba937bf13d40fb24,2016-01-14,32.0
2,air_ba937bf13d40fb24,2016-01-15,29.0
3,air_ba937bf13d40fb24,2016-01-16,22.0
4,air_ba937bf13d40fb24,2016-01-18,6.0


In [119]:
df_test.head(5)

,air_store_id,visit_date
0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9,2017-04-27


In [120]:
# =============================================================================
# handle datetime (no clock info)
# =============================================================================
print('seperating date time features ...')

time_format = '%Y-%m-%d'
def seperate_date(data):     
    # split date feature in real visit datetime
    data_time = pd.to_datetime(data.visit_date, format=time_format)
    data['Year_visit']= data_time.dt.year
    data['Month_visit'] = data_time.dt.month
    data['DayOfYear_visit'] = data_time.dt.dayofyear
    # data['DayOfMonth_visit'] = data_time.dt.day
#    data['WeekOfYear_visit'] = data_time.dt.week
    data['DayOfWeek_visit'] = data_time.dt.dayofweek
#    del data['visit_date']
    return data

seperate_date(df_av)
seperate_date(df_test)


seperating date time features ...


,air_store_id,visit_date,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit
0,air_00a91d42b08b08d9,2017-04-23,2017,4,113,6
1,air_00a91d42b08b08d9,2017-04-24,2017,4,114,0
2,air_00a91d42b08b08d9,2017-04-25,2017,4,115,1
3,air_00a91d42b08b08d9,2017-04-26,2017,4,116,2
4,air_00a91d42b08b08d9,2017-04-27,2017,4,117,3
...,...,...,...,...,...,...
32014,air_fff68b929994bfbd,2017-05-27,2017,5,147,5
32015,air_fff68b929994bfbd,2017-05-28,2017,5,148,6
32016,air_fff68b929994bfbd,2017-05-29,2017,5,149,0
32017,air_fff68b929994bfbd,2017-05-30,2017,5,150,1


In [121]:
# ------------------------------------------------------------------------------
time_format = "%Y-%m-%d %H:%M:%S"
def seperate_date(data):
    # split date feature in reservation datetime
    data_time = pd.to_datetime(data.reserve_datetime, format=time_format)
    data['Year_re']= data_time.dt.year
    data['Month_re'] = data_time.dt.month
    data['DayOfYear_re'] = data_time.dt.dayofyear
    # data['DayOfMonth_re'] = data_time.dt.day
#    data['WeekOfYear_re'] = data_time.dt.week
    data['DayOfWeek_re'] = data_time.dt.dayofweek
    data['Hour_re'] = data_time.dt.hour
#    del data['reserve_datetime']
    return data

seperate_date(df_ar)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016,1,1,4,19
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016,1,1,4,19
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016,1,1,4,16
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016,1,1,4,1
...,...,...,...,...,...,...,...,...,...
92373,air_db4b38ebe7a7ceff,2017-05-29 18:00:00,2017-04-17 19:00:00,6,2017,4,107,0,19
92374,air_1033310359ceeac1,2017-05-30 23:00:00,2017-04-19 17:00:00,8,2017,4,109,2,17
92375,air_877f79706adbfb06,2017-05-31 20:00:00,2017-04-12 23:00:00,3,2017,4,102,2,23
92376,air_900d755ebd2f7bbd,2017-05-31 20:00:00,2017-04-19 16:00:00,10,2017,4,109,2,16


In [122]:
def seperate_date(data):
    # split date feature in reservation datetime
    data_time = pd.to_datetime(data.reserve_datetime, format=time_format)
    data['Year_re_h']= data_time.dt.year
    data['Month_re_h'] = data_time.dt.month
    data['DayOfYear_re_h'] = data_time.dt.dayofyear
    # data['DayOfMonth_re_h'] = data_time.dt.day
#    data['WeekOfYear_re_h'] = data_time.dt.week
    data['DayOfWeek_re_h'] = data_time.dt.dayofweek
    data['Hour_re_h'] = data_time.dt.hour
#    del data['reserve_datetime']
    return data

seperate_date(df_hr)

,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h
103,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926,2016,1,1,4,15
121,2016-01-01 19:00:00,2016-01-01 16:00:00,2,air_08cb3c4ee6cd6a22,2016,1,1,4,16
272,2016-01-02 17:00:00,2016-01-01 22:00:00,3,air_6b15edd1b4fbb96a,2016,1,1,4,22
348,2016-01-02 18:00:00,2016-01-02 17:00:00,2,air_37189c92b6c761ec,2016,1,2,5,17
349,2016-01-02 18:00:00,2016-01-01 20:00:00,2,air_37189c92b6c761ec,2016,1,1,4,20
...,...,...,...,...,...,...,...,...,...
1999771,2017-05-26 18:00:00,2017-04-11 08:00:00,2,air_97159fc4e90053fe,2017,4,101,1,8
1999798,2017-05-26 19:00:00,2017-04-17 21:00:00,3,air_db1233ad855b34d5,2017,4,107,0,21
2000092,2017-05-27 19:00:00,2017-04-09 21:00:00,2,air_cb7467aed805e7fe,2017,4,99,6,21
2000095,2017-05-27 19:00:00,2017-04-13 13:00:00,6,air_6b15edd1b4fbb96a,2017,4,103,3,13


In [123]:
time_format = "%Y-%m-%d %H:%M:%S"
def seperate_date(data):
    # split date feature in reserved visiting datetime
    data_time = pd.to_datetime(data.visit_datetime, format=time_format)
    data['Year_re_visit']= data_time.dt.year
    data['Month_re_visit'] = data_time.dt.month
    data['DayOfYear_re_visit'] = data_time.dt.dayofyear
    # data['DayOfMonth_re_visit'] = data_time.dt.day
#    data['WeekOfYear_re_visit'] = data_time.dt.week
    data['DayOfWeek_re_visit'] = data_time.dt.dayofweek
    data['Hour_re_visit'] = data_time.dt.hour
#    del data['visit_datetime']
    return data

seperate_date(df_ar)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16,2016,1,1,4,19
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016,1,1,4,19,2016,1,1,4,19
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016,1,1,4,19,2016,1,1,4,19
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016,1,1,4,16,2016,1,1,4,20
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016,1,1,4,1,2016,1,1,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92373,air_db4b38ebe7a7ceff,2017-05-29 18:00:00,2017-04-17 19:00:00,6,2017,4,107,0,19,2017,5,149,0,18
92374,air_1033310359ceeac1,2017-05-30 23:00:00,2017-04-19 17:00:00,8,2017,4,109,2,17,2017,5,150,1,23
92375,air_877f79706adbfb06,2017-05-31 20:00:00,2017-04-12 23:00:00,3,2017,4,102,2,23,2017,5,151,2,20
92376,air_900d755ebd2f7bbd,2017-05-31 20:00:00,2017-04-19 16:00:00,10,2017,4,109,2,16,2017,5,151,2,20


In [124]:
def seperate_date(data):
    # split date feature in reserved visiting datetime
    data_time = pd.to_datetime(data.visit_datetime, format=time_format)
    data['Year_re_visit_h']= data_time.dt.year
    data['Month_re_visit_h'] = data_time.dt.month
    data['DayOfYear_re_visit_h'] = data_time.dt.dayofyear
    # data['DayOfMonth_re_visit_h'] = data_time.dt.day
    data['WeekOfYear_re_visit_h'] = data_time.dt.week
    data['DayOfWeek_re_visit_h'] = data_time.dt.dayofweek
    data['Hour_re_visit_h'] = data_time.dt.hour
#    del data['visit_datetime']
    return data

seperate_date(df_hr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h,Year_re_visit_h,Month_re_visit_h,DayOfYear_re_visit_h,WeekOfYear_re_visit_h,DayOfWeek_re_visit_h,Hour_re_visit_h
103,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926,2016,1,1,4,15,2016,1,1,53,4,19
121,2016-01-01 19:00:00,2016-01-01 16:00:00,2,air_08cb3c4ee6cd6a22,2016,1,1,4,16,2016,1,1,53,4,19
272,2016-01-02 17:00:00,2016-01-01 22:00:00,3,air_6b15edd1b4fbb96a,2016,1,1,4,22,2016,1,2,53,5,17
348,2016-01-02 18:00:00,2016-01-02 17:00:00,2,air_37189c92b6c761ec,2016,1,2,5,17,2016,1,2,53,5,18
349,2016-01-02 18:00:00,2016-01-01 20:00:00,2,air_37189c92b6c761ec,2016,1,1,4,20,2016,1,2,53,5,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999771,2017-05-26 18:00:00,2017-04-11 08:00:00,2,air_97159fc4e90053fe,2017,4,101,1,8,2017,5,146,21,4,18
1999798,2017-05-26 19:00:00,2017-04-17 21:00:00,3,air_db1233ad855b34d5,2017,4,107,0,21,2017,5,146,21,4,19
2000092,2017-05-27 19:00:00,2017-04-09 21:00:00,2,air_cb7467aed805e7fe,2017,4,99,6,21,2017,5,147,21,5,19
2000095,2017-05-27 19:00:00,2017-04-13 13:00:00,6,air_6b15edd1b4fbb96a,2017,4,103,3,13,2017,5,147,21,5,19


In [125]:
print('seperating date time features done! ...')
gc.collect()
print("=========================================================================================")

seperating date time features done! ...


In [126]:
le = preprocessing.LabelEncoder()
# =============================================================================
# label encoding
# =============================================================================
print('label encoding ...')

le.fit(df_as['air_genre_name'])
df_as['air_genre_name'] = le.fit_transform(df_as['air_genre_name'])

le.fit(df_as['air_area_name'])
df_as['air_area_name'] = le.fit_transform(df_as['air_area_name'])

le.fit(df_hs['hpg_genre_name'])
df_hs['hpg_genre_name'] = le.fit_transform(df_hs['hpg_genre_name'])

le.fit(df_hs['hpg_area_name'])
df_hs['hpg_area_name'] = le.fit_transform(df_hs['hpg_area_name'])


label encoding ...


In [127]:
df_hs.head(5)

,hpg_genre_name,hpg_area_name,latitude,longitude,air_store_id
98,9,20,35.668600,139.763043,air_2aab19554f91ff82
150,9,29,35.704960,139.828642,air_258ad2619d7bff9a
178,9,7,34.392106,132.461914,air_c47aa7493b15f297
216,9,32,34.669514,135.501425,air_96005f79124e12bf
351,9,30,35.711353,139.782684,air_f2c5a1f24279c531


In [128]:
le.fit(df_as['air_store_id'])


df_ar['air_store_id'] = le.transform(df_ar['air_store_id'])
df_as['air_store_id'] = le.transform(df_as['air_store_id'])
df_av['air_store_id'] = le.transform(df_av['air_store_id'])
df_hr['air_store_id'] = le.transform(df_hr['air_store_id'])
df_hs['air_store_id'] = le.transform(df_hs['air_store_id'])

df_test['air_store_id'] = le.transform(df_test['air_store_id'])


In [129]:
df_test.head(5)

,air_store_id,visit_date,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit
0,0,2017-04-23,2017,4,113,6
1,0,2017-04-24,2017,4,114,0
2,0,2017-04-25,2017,4,115,1
3,0,2017-04-26,2017,4,116,2
4,0,2017-04-27,2017,4,117,3


In [130]:
print('label encoding done !')
gc.collect()
print("=========================================================================================")

label encoding done !


In [131]:
# =============================================================================
# Merge dataset
# =============================================================================
features_to_drop = [
        'air_store_id__'
        ]

def merge_df(data, data_to_join):
    # merge dataframes        
    data = data.join(data_to_join, on='air_store_id', rsuffix='__', how='left')   
    return data

def fix_data(data):
    # drop __ data    
    for feature in features_to_drop:
        del data[feature]
    return data


In [132]:
df_av.head(5)

,air_store_id,visit_date,visitors,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit
0,603,2016-01-13,25.0,2016,1,13,2
1,603,2016-01-14,32.0,2016,1,14,3
2,603,2016-01-15,29.0,2016,1,15,4
3,603,2016-01-16,22.0,2016,1,16,5
4,603,2016-01-18,6.0,2016,1,18,0


In [133]:
df_ar.head(5)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit
0,426,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16,2016,1,1,4,19
1,711,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016,1,1,4,19,2016,1,1,4,19
2,711,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016,1,1,4,19,2016,1,1,4,19
3,426,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016,1,1,4,16,2016,1,1,4,20
4,712,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016,1,1,4,1,2016,1,1,4,20


In [134]:
df_as.head(5)

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,42,6,27,34.695124,135.197852
1,390,6,27,34.695124,135.197852
2,825,6,27,34.695124,135.197852
3,527,6,27,34.695124,135.197852
4,412,6,62,35.658068,139.751599


In [135]:
# Merge to df_train
print('merging dataframes ...')
df_train = merge_df(df_av, df_ar)
df_train = merge_df(df_train, df_as)

merging dataframes ...


In [136]:
df_train.head(5)

,air_store_id,visit_date,visitors,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit,air_store_id__,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit,air_store_id__,air_genre_name,air_area_name,latitude,longitude
0,603,2016-01-13,25.0,2016,1,13,2,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
1,603,2016-01-14,32.0,2016,1,14,3,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
2,603,2016-01-15,29.0,2016,1,15,4,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
3,603,2016-01-16,22.0,2016,1,16,5,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
4,603,2016-01-18,6.0,2016,1,18,0,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694


In [137]:
df_hr['reserve_visitors_hr'] = df_hr['reserve_visitors'] 
del df_hr['reserve_visitors']

In [138]:
df_hs['latitude_hr'] = df_hs['latitude'] 
del df_hs['latitude'] 

df_hs['longitude_hr'] = df_hs['longitude'] 
del df_hs['longitude'] 

In [139]:
df_hs.head(5)

,hpg_genre_name,hpg_area_name,air_store_id,latitude_hr,longitude_hr
98,9,20,124,35.668600,139.763043
150,9,29,102,35.704960,139.828642
178,9,7,637,34.392106,132.461914
216,9,32,481,34.669514,135.501425
351,9,30,793,35.711353,139.782684


In [140]:
df_hr.head(5)

,visit_datetime,reserve_datetime,air_store_id,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h,Year_re_visit_h,Month_re_visit_h,DayOfYear_re_visit_h,WeekOfYear_re_visit_h,DayOfWeek_re_visit_h,Hour_re_visit_h,reserve_visitors_hr
103,2016-01-01 19:00:00,2016-01-01 15:00:00,712,2016,1,1,4,15,2016,1,1,53,4,19,4
121,2016-01-01 19:00:00,2016-01-01 16:00:00,27,2016,1,1,4,16,2016,1,1,53,4,19,2
272,2016-01-02 17:00:00,2016-01-01 22:00:00,336,2016,1,1,4,22,2016,1,2,53,5,17,3
348,2016-01-02 18:00:00,2016-01-02 17:00:00,162,2016,1,2,5,17,2016,1,2,53,5,18,2
349,2016-01-02 18:00:00,2016-01-01 20:00:00,162,2016,1,1,4,20,2016,1,2,53,5,18,2


In [141]:
df_train.head(5)

,air_store_id,visit_date,visitors,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit,air_store_id__,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit,air_store_id__,air_genre_name,air_area_name,latitude,longitude
0,603,2016-01-13,25.0,2016,1,13,2,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
1,603,2016-01-14,32.0,2016,1,14,3,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
2,603,2016-01-15,29.0,2016,1,15,4,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
3,603,2016-01-16,22.0,2016,1,16,5,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694
4,603,2016-01-18,6.0,2016,1,18,0,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694


In [142]:
df_train = merge_df(df_train, df_hs)
df_train = merge_df(df_train, df_hr)

In [143]:
df_train.head(5)

,air_store_id,visit_date,visitors,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit,air_store_id__,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit,air_store_id__,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name,air_store_id__,latitude_hr,longitude_hr,visit_datetime__,reserve_datetime__,air_store_id__,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h,Year_re_visit_h,Month_re_visit_h,DayOfYear_re_visit_h,WeekOfYear_re_visit_h,DayOfWeek_re_visit_h,Hour_re_visit_h,reserve_visitors_hr
0,603,2016-01-13,25.0,2016,1,13,2,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,603,2016-01-14,32.0,2016,1,14,3,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,603,2016-01-15,29.0,2016,1,15,4,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,603,2016-01-16,22.0,2016,1,16,5,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,603,2016-01-18,6.0,2016,1,18,0,349,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,400,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
gc.collect()
fix_data(df_train)

,air_store_id,visit_date,visitors,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name,latitude_hr,longitude_hr,visit_datetime__,reserve_datetime__,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h,Year_re_visit_h,Month_re_visit_h,DayOfYear_re_visit_h,WeekOfYear_re_visit_h,DayOfWeek_re_visit_h,Hour_re_visit_h,reserve_visitors_hr
0,603,2016-01-13,25.0,2016,1,13,2,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,603,2016-01-14,32.0,2016,1,14,3,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,603,2016-01-15,29.0,2016,1,15,4,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,603,2016-01-16,22.0,2016,1,16,5,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,603,2016-01-18,6.0,2016,1,18,0,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,98,2017-04-18,6.0,2017,4,108,1,2016-01-06 12:00:00,2016-01-06 11:00:00,3,2016,1,6,2,11,2016,1,6,2,12,6,30,34.737597,135.341564,9.0,20.0,35.6686,139.763043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252104,98,2017-04-19,6.0,2017,4,109,2,2016-01-06 12:00:00,2016-01-06 11:00:00,3,2016,1,6,2,11,2016,1,6,2,12,6,30,34.737597,135.341564,9.0,20.0,35.6686,139.763043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252105,98,2017-04-20,7.0,2017,4,110,3,2016-01-06 12:00:00,2016-01-06 11:00:00,3,2016,1,6,2,11,2016,1,6,2,12,6,30,34.737597,135.341564,9.0,20.0,35.6686,139.763043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252106,98,2017-04-21,8.0,2017,4,111,4,2016-01-06 12:00:00,2016-01-06 11:00:00,3,2016,1,6,2,11,2016,1,6,2,12,6,30,34.737597,135.341564,9.0,20.0,35.6686,139.763043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
# Merge to df_test

df_test = merge_df(df_test, df_ar)
df_test = merge_df(df_test, df_as)

df_test = merge_df(df_test, df_hs)
df_test = merge_df(df_test, df_hr)
gc.collect()
fix_data(df_test)

,air_store_id,visit_date,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name,latitude_hr,longitude_hr,visit_datetime__,reserve_datetime__,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h,Year_re_visit_h,Month_re_visit_h,DayOfYear_re_visit_h,WeekOfYear_re_visit_h,DayOfWeek_re_visit_h,Hour_re_visit_h,reserve_visitors_hr
0,0,2017-04-23,2017,4,113,6,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16,2016,1,1,4,19,6,27,34.695124,135.197852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2017-04-24,2017,4,114,0,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16,2016,1,1,4,19,6,27,34.695124,135.197852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2017-04-25,2017,4,115,1,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16,2016,1,1,4,19,6,27,34.695124,135.197852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2017-04-26,2017,4,116,2,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16,2016,1,1,4,19,6,27,34.695124,135.197852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2017-04-27,2017,4,117,3,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016,1,1,4,16,2016,1,1,4,19,6,27,34.695124,135.197852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32014,828,2017-05-27,2017,5,147,5,2016-01-15 19:00:00,2016-01-09 13:00:00,5,2016,1,9,5,13,2016,1,15,4,19,9,23,43.055460,141.340956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32015,828,2017-05-28,2017,5,148,6,2016-01-15 19:00:00,2016-01-09 13:00:00,5,2016,1,9,5,13,2016,1,15,4,19,9,23,43.055460,141.340956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32016,828,2017-05-29,2017,5,149,0,2016-01-15 19:00:00,2016-01-09 13:00:00,5,2016,1,9,5,13,2016,1,15,4,19,9,23,43.055460,141.340956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32017,828,2017-05-30,2017,5,150,1,2016-01-15 19:00:00,2016-01-09 13:00:00,5,2016,1,9,5,13,2016,1,15,4,19,9,23,43.055460,141.340956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
print('merging dataframes done!')
gc.collect()
print("=========================================================================================")

merging dataframes done!


In [147]:
df_di.head(5)

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [148]:
# =============================================================================
# add holiday feature (for the visiting day)
# =============================================================================
df_di['visit_date'] = df_di['calendar_date']
del df_di['calendar_date'] 

In [149]:
def add_is_holiday(data):
    # merge dataframes        
    data = data.merge(df_di, on='visit_date', how='left')
    del data['day_of_week']
    return data

In [150]:
df_train = add_is_holiday(df_train)
df_test = add_is_holiday(df_test)

In [151]:
df_train.head(5)

,air_store_id,visit_date,visitors,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit,visit_datetime,reserve_datetime,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name,latitude_hr,longitude_hr,visit_datetime__,reserve_datetime__,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h,Year_re_visit_h,Month_re_visit_h,DayOfYear_re_visit_h,WeekOfYear_re_visit_h,DayOfWeek_re_visit_h,Hour_re_visit_h,reserve_visitors_hr,holiday_flg
0,603,2016-01-13,25.0,2016,1,13,2,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,603,2016-01-14,32.0,2016,1,14,3,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,603,2016-01-15,29.0,2016,1,15,4,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,603,2016-01-16,22.0,2016,1,16,5,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,603,2016-01-18,6.0,2016,1,18,0,2016-01-13 11:00:00,2016-01-06 17:00:00,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [152]:
# =============================================================================
# drop date-time-hour info
# =============================================================================
def drop_datetime_info(data):
    del data['visit_date'], data['visit_datetime'], data['reserve_datetime'], data['visit_datetime__'], data['reserve_datetime__']
#    del data['visit_date'], data['visit_datetime'], data['reserve_datetime']
    return data
df_train = drop_datetime_info(df_train)

def drop_datetime_info(data):
    del data['visit_date'], data['visit_datetime'], data['reserve_datetime'], data['visit_datetime__'], data['reserve_datetime__']
#    del data['visit_date'], data['visit_datetime'], data['reserve_datetime']
    return data
df_test = drop_datetime_info(df_test)

In [153]:
# =============================================================================
# autoclean
# =============================================================================
#df_train_clean = autoclean(df_train)
#df_test_clean = autoclean(df_test)
#
train = df_train.fillna(-1)
test = df_test.fillna(-1)

In [154]:
train.head(5)

,air_store_id,visitors,Year_visit,Month_visit,DayOfYear_visit,DayOfWeek_visit,reserve_visitors,Year_re,Month_re,DayOfYear_re,DayOfWeek_re,Hour_re,Year_re_visit,Month_re_visit,DayOfYear_re_visit,DayOfWeek_re_visit,Hour_re_visit,air_genre_name,air_area_name,latitude,longitude,hpg_genre_name,hpg_area_name,latitude_hr,longitude_hr,Year_re_h,Month_re_h,DayOfYear_re_h,DayOfWeek_re_h,Hour_re_h,Year_re_visit_h,Month_re_visit_h,DayOfYear_re_visit_h,WeekOfYear_re_visit_h,DayOfWeek_re_visit_h,Hour_re_visit_h,reserve_visitors_hr,holiday_flg
0,603,25.0,2016,1,13,2,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
1,603,32.0,2016,1,14,3,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
2,603,29.0,2016,1,15,4,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
3,603,22.0,2016,1,16,5,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0
4,603,6.0,2016,1,18,0,2,2016,1,6,2,17,2016,1,13,2,11,7,2,33.585816,130.398694,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0


# Split the train data for modeling

In [155]:
x_train = train.drop(['visitors'], axis=1)
y_train = np.log1p(train['visitors'].values)

# LightGBM with optuna

In [156]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(x_train,y_train,test_size=0.2,random_state=42)

In [157]:
#LightGBM用のデータ作成
lgb_train = lgb.Dataset(X_train, Y_train)
lgb_valid = lgb.Dataset(X_test, Y_test, reference=lgb_train)

In [158]:
# to record eval results for plotting
evals_result = {} 

In [159]:
#OptunaチューニングParam
optuna_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'mean_squared_error',# regression(回帰), binary(二値分類, ラベルは0,1),multiclass(多クラス分類, metricとしては、multi_logloss, multi_error(正答率))
    'metric': "regression", #regression : 絶対値誤差関数(L1)ならばmae, 2乗誤差関数(L2)ならばmseと指定, rmse
                      #binary : binary_logloss(クロスエントロピー)とbinary_error(正答率)の2つ
                      #multiclass : multi_logloss(softmax関数)とmulti_error(正答率)の2つ
    #'num_leaves': 20,#木にある分岐の個数．デフォルトは31, 大きくすると精度は上がるが過学習が進む
    #'max_depth': 10,#木構造の深さを限定するための変数．データが少ないときに過学習を防ぐために設定する．デフォルト値は-1で0以下の値は制限なしを意味する．
    #'learning_rate': 0.05,#学習率。デフォルトは0.1
    #'feature_fraction': 0.9,#学習の高速化と過学習の抑制に使用される．データの特徴量のfeature_fraction * 100 % だけ使用する．
    #'min_data_in_leaf':4,#大きくすることでデータと分岐が1対1対応することを防ぎ，結果として過学習を抑制する．
    #'verbose':50
}

In [160]:
#Optunaを用いて最適化し、LGBにTrainさせる
model = lgb.train(optuna_params,
                  lgb_train,
                  valid_sets=lgb_valid,
                  verbose_eval=0,
                  num_boost_round=1000,
                  early_stopping_rounds=50
                  )

In [161]:
y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)

In [162]:
# eval rmse
print('\nThe rmse of prediction is:', mean_squared_error(Y_test, y_pred_test) ** 0.5)
accuracy = mean_squared_error(Y_test, y_pred_test) ** 0.5


The rmse of prediction is: 0.5270059188240132


In [163]:
best_params = model.params
print("Best params:", best_params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))


Best params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mean_squared_error', 'metric': 'regression'}
  Accuracy = 0.5270059188240132
  Params: 
    task: train
    boosting_type: gbdt
    objective: mean_squared_error
    metric: regression


In [164]:
y_pred = model.predict(test, num_iteration=model.best_iteration)

In [165]:
y_pred = np.expm1(y_pred)

In [166]:
df_ss

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0
...,...,...
32014,air_fff68b929994bfbd_2017-05-27,0
32015,air_fff68b929994bfbd_2017-05-28,0
32016,air_fff68b929994bfbd_2017-05-29,0
32017,air_fff68b929994bfbd_2017-05-30,0


# Submit

In [167]:
sub = pd.DataFrame(pd.read_csv('sample_submission.csv.zip')['id'])
sub['visitors'] = list(map(int, y_pred))
sub.to_csv('submission_yshimamu.csv', index=False)

In [168]:
!kaggle competitions submit recruit-restaurant-visitor-forecasting -f submission_yshimamu.csv -m "First commit"

100% 1.06M/1.06M [00:05<00:00, 209kB/s]
Successfully submitted to Recruit Restaurant Visitor Forecasting